## Customer Churn Analysis

# The story Behind The Data
A bank is concerned that more and more customers are leaving its credit card services. They would really appreciate if someone could analyze it for them, in order to understand the main reasons for leaving the services, and to come up with recommendations for how the bank can mitigate that. Eventually, the bank would like to proactively implement these recommendations in order to keep their customers happy.

**A full ERD can be found [here](https://dbdiagram.io/d/638cdd8abae3ed7c45449eed)**

# Data Description
In this task, few datasets are provided:

1. **`BankChurners.csv`**   - this file contains basic information about each client (10 columns). The columns are:
    - `CLIENTNUM` - Client number. Unique identifier for the customer holding the account;
    - `Attrition Flag` - Internal event (customer activity) variable - if the client had churned (attrited) or not (existing).
    - `Dependent Count` - Demographic variable - Number of dependents
    - `Card_Category` - Product Variable - Type of Card (Blue, Silver, Gold, Platinum)
    - `Months_on_book` - Period of relationship with bank
    - `Months_Inactive_12_mon` - No. of months inactive in the last 12 months
    - `Contacts_Count_12_mon` - No. of Contacts in the last 12 months
    - `Credit_Limit` - Credit Limit on the Credit Card
    - `Avg_Open_To_Buy` - Open to Buy Credit Line (Average of last 12 months)
    - `Avg_Utilization_Ratio` - Average Card Utilization Ratio
    
    
2. **`basic_client_info.csv`** - this file contains some basic client info per each client
(6 columns) -
    - `CLIENTNUM` - Client number. Unique identifier for the customer holding the account
    - `Customer Age` - Demographic variable - Customer's Age in Years
    - `Gender` - Demographic variable - M=Male, F=Female
    - `Education_Level` - Demographic variable - Educational Qualification of the account holder (example: high school, college graduate, etc.`
    - `Marital_Status` - Demographic variable - Married, Single, Divorced, Unknown
    - `Income_Category` - Demographic variable - Annual Income Category of the account holder (< $40K, $40K - 60K, $60K - $80K, $80K-$120K, > $120K, Unknown)
    
3. **`enriched_churn_data.csv`** - this file contains some enriched data about each client (7 columns) -
    - `CLIENTNUM` - Client number. Unique identifier for the customer holding the account
    - `Total_Relationship_Count` - Total no. of products held by the customer
    - `Total_Revolving_Bal` - Total Revolving Balance on the Credit Card
    - `Total_Amt_Chng_Q4_Q1` - Change in Transaction Amount (Q4 over Q1)
    - `Total_Trans_Amt` - Total Transaction Amount (Last 12 months)
    - `Total_Trans_Ct` - Total Transaction Count (Last 12 months)
    - `Total_Ct_Chng_Q4_Q1` - Change in Transaction Count (Q4 over Q1)

In [1]:
# import necessary libraries
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
pio.renderers.default = 'svg'
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

### Data wrangling

In [2]:
# import from csv
data0 = pd.read_csv('bankchurners.csv')
data1 = pd.read_csv('enriched_churn_data.csv')
data2 = pd.read_csv('basic_client_info.csv')

In [3]:
# merge all tables
data01 = pd.merge(data0, data1, on='clientnum')
data = pd.merge(data01, data2, on='clientnum')
df = pd.DataFrame(data=data)

In [4]:
pd.set_option('max_colwidth', 10)
pd.set_option('display.max_columns', 21)
df.sample(10)

In [5]:
df.info()

### Data Cleaning

In [6]:
#make a copy
df_clean = df.copy()

In [7]:
#check duplicates
df_clean.duplicated().sum()

In [8]:
#check nulls
df_clean.attrition_flag.isna().sum()

In [9]:
## Pre-processing

In [10]:
cat_cols = [c for c in df_clean.columns if df_clean[c].dtype == 'object']
cat_cols               

In [11]:
num_cols = [n for n in df_clean.columns if df_clean[n].dtype != 'object']
num_cols  

In [12]:
df_clean[num_cols].describe()

In [13]:
df_clean.attrition_flag.value_counts()  #below the anual churning rate?

In [14]:
# Define a dictionary mapping the categories to binary values
dict_ch = {
    'Existing Customer': 0,
    'Attrited Customer': 1,
}

# Use the map() method to replace the values in the 'attrition_flag' column
df_clean['churned'] = df_clean['attrition_flag'].map(dict_ch)

In [15]:
# Drop the original 'attrition_flag' column
df_clean.drop('attrition_flag', axis=1, inplace=True)

In [16]:
df_clean

In [17]:
# Calculate the percentage of churned customers
num_churned = df_clean[df_clean["churned"] == 1].shape[0]
num_total = df_clean.shape[0]
percent_churned = (num_churned / num_total) * 100

# Format the result as a percentage
print("Percentage of churned customers: {:.2f}%".format(percent_churned))

#### Observations

The annual churn rate for credit cards can vary greatly depending on the type of card and the target demographic. On average, it is common to see an annual churn rate of around 20% for credit cards. However, some high-end, premium credit cards may have a lower churn rate, while cards targeted towards subprime borrowers may have a higher churn rate.

It's important to note that a lower churn rate is typically seen as a positive sign for a credit card issuer, as it indicates that customers are satisfied with the card and less likely to switch to a different one. On the other hand, a higher churn rate can indicate that customers are not satisfied with the card or are switching to a different one with more attractive terms.

A preliminary overview of the dataset shows that around 83.93% of customers are active, while 16.07% have churned.

In [18]:
#Convert 'attrition_flag'to boolean dtype
dict_g = {
    'F': ['Female'],
    'M': [' Male'],
}
df_clean['gender'] = df_clean['gender']
df_clean.replace({"gender": dict_g}, inplace=True)

In [19]:
df.describe(include='object')

In [20]:
df_clean.describe().loc[['min', 'max', 'mean']].round(2)

In [21]:
df_clean.info()

In [22]:
corr_matrix = df_clean.corr(numeric_only=True)

fig = px.imshow(corr_matrix,
color_continuous_scale='YlOrRd',
aspect="auto",
title='Correlation Matrix')
fig.update_traces(
text=corr_matrix.round(2),
texttemplate="%{text:.2f}",
textfont_size=12,
hovertemplate=
"Feature 1: %{y}<br>Feature 2: %{x}<br>Correlation: %{text:.2f}<extra></extra>",
customdata=np.moveaxis(corr_matrix.values, 0, -1))
fig.update_xaxes(tickangle=90, tickfont=dict(size=12), title="")
fig.update_yaxes(tickfont=dict(size=12), title="")
fig.update_coloraxes(showscale=True, colorbar=dict(len=0.4, y=0.75))
fig.show()

In [23]:
# filter for churned
df_churned = df_clean[df_clean['churned'] == 1]

corr_matrix = df_churned.corr(numeric_only=True)

fig = px.imshow(corr_matrix,
                color_continuous_scale='YlOrRd',
                aspect="auto",
                title='Correlation Matrix for Churned')
fig.update_traces(
    text=corr_matrix.round(2),
    texttemplate="%{text:.2f}",
    textfont_size=12,
    hovertemplate=
    "Feature 1: %{y}<br>Feature 2: %{x}<br>Correlation: %{text:.2f}<extra></extra>",
    customdata=np.moveaxis(corr_matrix.values, 0, -1))
fig.update_xaxes(tickangle=90, tickfont=dict(size=12), title="")
fig.update_yaxes(tickfont=dict(size=12), title="")
fig.update_coloraxes(showscale=True, colorbar=dict(len=0.4, y=0.75))
fig.show()

In [24]:
X = df_clean.drop('churned', axis=1)
y = df_clean['churned']

# One-hot encode categorical variables
X_encoded = pd.get_dummies(X,
                           columns=[
                               'card_category', 'gender', 'education_level',
                               'marital_status', 'income_category'
                           ])

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

# Train the XGBoost model
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

# Evaluate the model on the testing set
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print('Accuracy:', accuracy)
print('Confusion matrix:', confusion_mat)

# Visualize the model's feature importances
importances = pd.Series(xgb_model.feature_importances_,
                        index=X_encoded.columns)
importances_nonzero = importances[importances != 0].sort_values(ascending=True)

# Create a horizontal bar chart of feature importances
fig = go.Figure(
    go.Bar(x=importances_nonzero.values,
           y=importances_nonzero.index,
           orientation='h'))

fig.update_layout(title='Feature Importances (XGBoost)',
                  xaxis_title='F score',
                  yaxis_title='Feature')

fig.show()

In [25]:
# Retrain the XGBoost model with the most important features
important_features = importances_nonzero.tail(10).index.tolist()
X_important = X_encoded[important_features]
X_train_important, X_test_important, y_train, y_test = train_test_split(
    X_important, y, test_size=0.2, random_state=42)
xgb_model_important = xgb.XGBClassifier()
xgb_model_important.fit(X_train_important, y_train)

# Evaluate the model on the testing set
y_pred = xgb_model_important.predict(X_test_important)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)
print('Confusion matrix:', confusion_mat)

In [26]:
# Create an XGBoost classifier with default hyperparameters
clf = xgb.XGBClassifier()

# Perform 5-fold cross-validation on the data
scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1')

# Print the average F1 score across all folds
print('Average F1 score:', scores.mean())

In [27]:
# initialize the logistic regression model
lr_model = LogisticRegression()

# train the model on the training data
lr_model.fit(X_train, y_train)

# make predictions on the testing data
y_pred = lr_model.predict(X_test)

# evaluate the model's performance
print(classification_report(y_test, y_pred, zero_division=1))

 - In the context of XGBoost, the F-score refers to the feature importance score calculated for each feature. The F-score is a measure of the total reduction in the impurity of the model when splitting on a particular feature. It is used to rank the importance of features in the model and can be used to identify the most relevant features for prediction. Features with higher F-scores are considered more important to the model.

In [28]:
# Create the DMatrix
dmatrix = xgb.DMatrix(data=X_encoded, label=y)

# Create the parameter dictionary: params
params = {"objective": "binary:logistic", "max_depth": 2}

# Train the model: xg_reg
xg_reg = xgb.train(params=params, dtrain=dmatrix, num_boost_round=10)

# Plot the first tree
fig = plt.figure(figsize=(25, 20), dpi=300)
xgb.plot_tree(xg_reg, num_trees=0, ax=plt.gca(), fontsize=14)
plt.show()

In [29]:
# Plot the fifth tree
fig = plt.figure(figsize=(25, 20), dpi=300)
xgb.plot_tree(xg_reg, num_trees=4, ax=plt.gca(), fontsize=14)
plt.show()

### Economics

In [30]:
# Plot distribution of total transaction count by attrition flag
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df_clean,
            x='total_trans_ct',
            hue='churned',
            fill=True,
            alpha=0.5,
            palette='cool')
plt.title('Distribution of Total Transaction Count by Attrition Flag')
plt.xlabel('Total Transaction Count')
plt.ylabel('Density')
plt.show()

In [31]:
churned_trans_by_gender_age = df_clean[df_clean['churned'] == 1].groupby(
    ['gender'])['total_trans_ct'].describe()
pd.set_option('max_colwidth', 20)
pd.set_option('display.max_columns', 10)
churned_trans_by_gender_age

In [32]:
# Plot distribution of total revolving balance by attrition flag
plt.figure(figsize=(10, 6), dpi=250)
sns.kdeplot(data=df_clean,
            x='total_revolving_bal',
            hue='churned',
            fill=True,
            alpha=0.5,
            palette='cool')
plt.title('Distribution of Total Revolving Balance by Attrition Flag')
plt.xlabel('Total Revolving Balance')
plt.ylabel('Density')
plt.show()

In [33]:
churned_bal_by_gender_age = df_clean[df_clean['churned'] == 1].groupby(
    ['gender'])['total_revolving_bal'].describe()
pd.set_option('max_colwidth', 20)
pd.set_option('display.max_columns', 10)
churned_bal_by_gender_age

In [34]:
# Plot distribution of total relationship count by attrition flag
plt.figure(figsize=(10, 6), dpi=250)
sns.kdeplot(data=df_clean,
            x='total_relationship_count',
            hue='churned',
            fill=True,
            alpha=0.5,
            palette='cool')
plt.title('Distribution of Total Relationship Count by Attrition Flag')
plt.xlabel('Total Relationship Count')
plt.ylabel('Density')
plt.show()

In [35]:
churned_count_by_relationship = df_clean[df_clean['churned'] == 1].groupby(
    ['total_relationship_count'])['clientnum'].count()
churned_count_by_relationship.describe()

In [36]:
# Plot distribution of total transaction amount by attrition flag
plt.figure(figsize=(10, 6), dpi=250)
sns.kdeplot(data=df_clean,
            x='total_trans_amt',
            hue='churned',
            fill=True,
            alpha=0.5,
            palette='cool')
plt.title('Distribution of Total Transaction Amount by Attrition Flag')
plt.xlabel('Total Transaction Amount')
plt.ylabel('Density')
plt.show()

In [37]:
churned_trans_by_gender_age = df_clean[df_clean['churned'] == 1].groupby(
    ['gender'])['total_trans_amt'].describe()

In [38]:
pd.set_option('max_colwidth', 20)
pd.set_option('display.max_columns', 10)
churned_trans_by_gender_age

In [39]:
# Plot distribution of total count change by attrition flag
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df_clean,
            x='total_ct_chng_q4_q1',
            hue='churned',
            fill=True,
            alpha=0.5,
            palette='cool')
plt.title('Distribution of Total Count Change by Attrition Flag')
plt.xlabel('Total Count Change Q4-Q1')
plt.ylabel('Density')
plt.show()

In [40]:
churned_total_ct_chng_q4_q1_by_gender = df_clean[
    df_clean['churned'] == 1].groupby(['gender'
                                       ])['total_ct_chng_q4_q1'].describe()
pd.set_option('max_colwidth', 20)
pd.set_option('display.max_columns', 10)
churned_total_ct_chng_q4_q1_by_gender

### Client Activity

In [41]:
# Set style
sns.set_style('whitegrid')

# Create distribution plot of contacts count by attrition flag
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df_clean,
            x='contacts_count_12_mon',
            hue='churned',
            fill=True,
            alpha=0.5,
            palette='cool')
plt.title(
    'Distribution of Contacts Count in the Last 12 Months by Attrition Flag')
plt.xlabel('Contacts Count in the Last 12 Months')
plt.ylabel('Density')
plt.show()

In [42]:
# Import libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')

# Create histogram of contacts count by attrition flag
plt.figure(figsize=(10, 6))
sns.histplot(data=df_clean,
            x='contacts_count_12_mon',
            hue='churned',
            multiple='stack',
            palette='cool')
plt.title('Distribution of Contacts Count in the Last 12 Months by Attrition Flag')
plt.xlabel('Contacts Count in the Last 12 Months')
plt.ylabel('Count')
plt.show()


In [43]:
# Plot distribution of months on book by attrition flag
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df_clean,
            x='months_on_book',
            hue='churned',
            fill=True,
            alpha=0.5,
            palette='cool')
plt.title('Distribution of Months on Book by Attrition Flag')
plt.xlabel('Months on Book')
plt.ylabel('Density')
plt.show()

 - Customer retention: Since the mean number of months on the book is 36, we can assume that a significant proportion of the customers have been with the bank for a relatively long period of time. This could indicate that the bank has been successful in retaining its customers over a longer period of time.